In [1]:
import json
import os
import numpy as np
import re
from tqdm import tqdm
from glob import glob
from collections import defaultdict
from datasets import Audio
import torchaudio
import torch
import soundfile as sf

In [2]:
sr = 16000
reader = Audio(sampling_rate = sr)

In [4]:
data = []
with open('/home/husein/ssd3/stt/dataset-whisper-stt-2023-09-04.jsonl') as fopen:
    for l in tqdm(fopen):
        data.append(json.loads(l))

2548584it [00:06, 395388.47it/s]


In [5]:
len(data)

2548584

In [6]:
probs = [d['avg_logprob'] for d in data]

In [7]:
np.percentile(probs, 80)

-0.39062501179675263

In [8]:
d = defaultdict(list)
for d_ in data:
    if d_['avg_logprob'] >= -0.38:
        k = f"{d_['no']}-{d_['i']}"
        d[k].append(d_)
        
len(d)

187137

In [9]:
def read(f):
    audio = reader.decode_example(reader.encode_example(f))['array']
    return audio, len(audio) / sr

In [10]:
!mkdir /home/husein/ssd3/postfilter-stt
!rm -rf /home/husein/ssd3/postfilter-stt/part2
!mkdir /home/husein/ssd3/postfilter-stt/part2

mkdir: cannot create directory ‘/home/husein/ssd3/postfilter-stt’: File exists


In [11]:
import string
printable = set(string.printable)

In [12]:
maxlen = 12

index = 0
directory = '/home/husein/ssd3/postfilter-stt/part2'
X, Y = [], []

for key in tqdm(sorted(list(d.keys()))):
    values = {d_['k']: d_ for d_ in d[key]}
    sorted_k = sorted(values.keys())
    temp = []
    temp_text = []
    idx = []
    lens = 0
    last = None
    
    for k in sorted_k:
        try:
            y, l = read(values[k]['file'])
            t = values[k]['cleaned']
            ori_t = values[k]['text']

            if len(re.findall(r'\d+', t)):
                continue

            if len(t.strip()) < 2:
                continue

            if len(set(ori_t) - printable):
                continue

            if last is None:
                temp.append(y)
                idx.append(k)
                temp_text.append(t)
                lens += l

            elif k - last != 1 or (lens + l) >= 12:
                audio_path = os.path.join(directory, f'{index}.mp3')
                a = np.concatenate(temp, axis = 0)
                sf.write(audio_path, a, 16000)
                X.append(audio_path)
                Y.append(' '.join(temp_text).strip())
                index += 1

                temp = [y]
                lens = l
                idx = [k]
                temp_text = [t]
            else:
                lens += l
                temp.append(y)
                idx.append(k)
                temp_text.append(t)

            last = k
        except Exception as e:
            print(e)

    if len(temp):
        audio_path = os.path.join(directory, f'{index}.mp3')
        a = np.concatenate(temp, axis = 0)
        sf.write(audio_path, a, 16000)
        X.append(audio_path)
        Y.append(' '.join(temp_text).strip())
        index += 1

100%|█████████████████████████████████| 187137/187137 [1:54:31<00:00, 27.23it/s]


In [13]:
total_lens = []
for f in tqdm(X):
    audio = reader.decode_example(reader.encode_example(f))['array']
    total_lens.append(len(audio) / sr)

100%|██████████████████████████████████| 204612/204612 [05:13<00:00, 652.93it/s]


In [14]:
len(X) / len(data)

0.08028458155587573

In [15]:
sum(total_lens) / 60 / 60

481.1661800000721

In [16]:
with open('postfilter-part-2023-09-04.json', 'w') as fopen:
    json.dump({'X': X, 'Y': Y}, fopen)